In [1]:
import math
import copy
from terminaltables import AsciiTable
import progressbar

In [2]:
import numpy as np

# Collection of activation functions
# Reference: https://en.wikipedia.org/wiki/Activation_function

class Sigmoid():
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))

    def gradient(self, x):
        return self.__call__(x) * (1 - self.__call__(x))



class ReLU():
    def __call__(self, x):
        return np.where(x >= 0, x, 0)

    def gradient(self, x):
        return np.where(x >= 0, 1, 0)




class Adam():
    def __init__(self, learning_rate=0.001, b1=0.9, b2=0.999):
        self.learning_rate = learning_rate
        self.eps = 1e-8
        self.m = None
        self.v = None
        # Decay rates
        self.b1 = b1
        self.b2 = b2

    def update(self, w, grad_wrt_w):
        # If not initialized
        if self.m is None:
            self.m = np.zeros(np.shape(grad_wrt_w))
            self.v = np.zeros(np.shape(grad_wrt_w))
        
        self.m = self.b1 * self.m + (1 - self.b1) * grad_wrt_w
        self.v = self.b2 * self.v + (1 - self.b2) * np.power(grad_wrt_w, 2)

        m_hat = self.m / (1 - self.b1)
        v_hat = self.v / (1 - self.b2)

        self.w_updt = self.learning_rate * m_hat / (np.sqrt(v_hat) + self.eps)

        return w - self.w_updt

In [3]:
def accuracy_score(y_true, y_pred):
    """ Compare y_true to y_pred and return the accuracy """
    accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)
    return accuracy

class Loss(object):
    def loss(self, y_true, y_pred):
        return NotImplementedError()

    def gradient(self, y, y_pred):
        raise NotImplementedError()

    def acc(self, y, y_pred):
        return 0
    
class Loss(object):
    def loss(self, y_true, y_pred):
        return NotImplementedError()

    def gradient(self, y, y_pred):
        raise NotImplementedError()

    def acc(self, y, y_pred):
        return 0

class CrossEntropy(Loss):
    def __init__(self): pass

    def loss(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - y * np.log(p) - (1 - y) * np.log(1 - p)

    def acc(self, y, p):
        return accuracy_score(np.argmax(y, axis=1), np.argmax(p, axis=1))

    def gradient(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - (y / p) + (1 - y) / (1 - p)

In [4]:
class Layer(object):

    def set_input_shape(self, shape):
        """ Sets the shape that the layer expects of the input in the forward
        pass method """
        self.input_shape = shape

    def layer_name(self):
        """ The name of the layer. Used in model summary. """
        return self.__class__.__name__

    def parameters(self):
        """ The number of trainable parameters used by the layer """
        return 0

    def forward_pass(self, X, training):
        """ Propogates the signal forward in the network """
        raise NotImplementedError()

    def backward_pass(self, accum_grad):
        """ Propogates the accumulated gradient backwards in the network.
        If the has trainable weights then these weights are also tuned in this method.
        As input (accum_grad) it receives the gradient with respect to the output of the layer and
        returns the gradient with respect to the output of the previous layer. """
        raise NotImplementedError()

    def output_shape(self):
        """ The shape of the output produced by forward_pass """
        raise NotImplementedError()
        

activation_functions = {
    'relu': ReLU,
    'sigmoid': Sigmoid

}
        
class Activation(Layer):
    """A layer that applies an activation operation to the input.

    Parameters:
    -----------
    name: string
        The name of the activation function that will be used.
    """

    def __init__(self, name):
        self.activation_name = name
        self.activation_func = activation_functions[name]()
        self.trainable = True

    def layer_name(self):
        return "Activation (%s)" % (self.activation_func.__class__.__name__)

    def forward_pass(self, X, training=True):
        self.layer_input = X
        return self.activation_func(X)

    def backward_pass(self, accum_grad):
        return accum_grad * self.activation_func.gradient(self.layer_input)

    def output_shape(self):
        return self.input_shape
    
class BatchNormalization(Layer):
    """Batch normalization.
    """
    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.trainable = True
        self.eps = 0.01
        self.running_mean = None
        self.running_var = None

    def initialize(self, optimizer):
        # Initialize the parameters
        self.gamma  = np.ones(self.input_shape)
        self.beta = np.zeros(self.input_shape)
        # parameter optimizers
        self.gamma_opt  = copy.copy(optimizer)
        self.beta_opt = copy.copy(optimizer)

    def parameters(self):
        return np.prod(self.gamma.shape) + np.prod(self.beta.shape)

    def forward_pass(self, X, training=True):

        # Initialize running mean and variance if first run
        if self.running_mean is None:
            self.running_mean = np.mean(X, axis=0)
            self.running_var = np.var(X, axis=0)

        if training and self.trainable:
            mean = np.mean(X, axis=0)
            var = np.var(X, axis=0)
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var
        else:
            mean = self.running_mean
            var = self.running_var

        # Statistics saved for backward pass
        self.X_centered = X - mean
        self.stddev_inv = 1 / np.sqrt(var + self.eps)

        X_norm = self.X_centered * self.stddev_inv
        output = self.gamma * X_norm + self.beta

        return output

    def backward_pass(self, accum_grad):

        # Save parameters used during the forward pass
        gamma = self.gamma

        # If the layer is trainable the parameters are updated
        if self.trainable:
            X_norm = self.X_centered * self.stddev_inv
            grad_gamma = np.sum(accum_grad * X_norm, axis=0)
            grad_beta = np.sum(accum_grad, axis=0)

            self.gamma = self.gamma_opt.update(self.gamma, grad_gamma)
            self.beta = self.beta_opt.update(self.beta, grad_beta)

        batch_size = accum_grad.shape[0]

        # The gradient of the loss with respect to the layer inputs (use weights and statistics from forward pass)
        accum_grad = (1 / batch_size) * gamma * self.stddev_inv * (
            batch_size * accum_grad
            - np.sum(accum_grad, axis=0)
            - self.X_centered * self.stddev_inv**2 * np.sum(accum_grad * self.X_centered, axis=0)
            )

        return accum_grad

    def output_shape(self):
        return self.input_shape

In [5]:


class Dense(Layer):
    """A fully-connected NN layer.
    Parameters:
    -----------
    n_units: int
        The number of neurons in the layer.
    input_shape: tuple
        The expected input shape of the layer. For dense layers a single digit specifying
        the number of features of the input. Must be specified if it is the first layer in
        the network.
    """
    def __init__(self, n_units, input_shape=None):
        self.layer_input = None
        self.input_shape = input_shape
        self.n_units = n_units
        self.trainable = True
        self.W = None
        self.w0 = None

    def initialize(self, optimizer):
        # Initialize the weights
        limit = 1 / math.sqrt(self.input_shape[0])
        self.W  = np.random.uniform(-limit, limit, (self.input_shape[0], self.n_units))
        self.w0 = np.zeros((1, self.n_units))
        # Weight optimizers
        self.W_opt  = copy.copy(optimizer)
        self.w0_opt = copy.copy(optimizer)

    def parameters(self):
        return np.prod(self.W.shape) + np.prod(self.w0.shape)

    def forward_pass(self, X, training=True):
        self.layer_input = X
        return X.dot(self.W) + self.w0

    def backward_pass(self, accum_grad):
        # Save weights used during forwards pass
        W = self.W

        if self.trainable:
            # Calculate gradient w.r.t layer weights
            grad_w = self.layer_input.T.dot(accum_grad)
            grad_w0 = np.sum(accum_grad, axis=0, keepdims=True)

            # Update the layer weights
            self.W = self.W_opt.update(self.W, grad_w)
            self.w0 = self.w0_opt.update(self.w0, grad_w0)

        # Return accumulated gradient for next layer
        # Calculated based on the weights used during the forward pass
        accum_grad = accum_grad.dot(W.T)
        return accum_grad

    def output_shape(self):
        return (self.n_units, )

In [6]:
def batch_iterator(X, y=None, batch_size=64):
    """ Simple batch generator """
    n_samples = X.shape[0]
    for i in np.arange(0, n_samples, batch_size):
        begin, end = i, min(i+batch_size, n_samples)
        if y is not None:
            yield X[begin:end], y[begin:end]
        else:
            yield X[begin:end]

In [7]:
bar_widgets = [
    'Training: ', progressbar.Percentage(), ' ', progressbar.Bar(marker="-", left="[", right="]"),
    ' ', progressbar.ETA()
]

class NeuralNetwork():
    """Neural Network. Deep Learning base model.

    Parameters:
    -----------
    optimizer: class
        The weight optimizer that will be used to tune the weights in order of minimizing
        the loss.
    loss: class
        Loss function used to measure the model's performance. SquareLoss or CrossEntropy.
    validation: tuple
        A tuple containing validation data and labels (X, y)
    """
    def __init__(self, optimizer, loss, validation_data=None):
        self.optimizer = optimizer
        self.layers = []
        self.errors = {"training": [], "validation": []}
        self.loss_function = loss()
        self.progressbar = progressbar.ProgressBar(widgets=bar_widgets)

        self.val_set = None
        if validation_data:
            X, y = validation_data
            self.val_set = {"X": X, "y": y}

    def set_trainable(self, trainable):
        """ Method which enables freezing of the weights of the network's layers. """
        for layer in self.layers:
            layer.trainable = trainable

    def add(self, layer):
        """ Method which adds a layer to the neural network """
        # If this is not the first layer added then set the input shape
        # to the output shape of the last added layer
        if self.layers:
            layer.set_input_shape(shape=self.layers[-1].output_shape())

        # If the layer has weights that needs to be initialized 
        if hasattr(layer, 'initialize'):
            layer.initialize(optimizer=self.optimizer)

        # Add layer to the network
        self.layers.append(layer)

    def test_on_batch(self, X, y):
        """ Evaluates the model over a single batch of samples """
        y_pred = self._forward_pass(X, training=False)
        loss = np.mean(self.loss_function.loss(y, y_pred))
        acc = self.loss_function.acc(y, y_pred)

        return loss, acc

    def train_on_batch(self, X, y):
        """ Single gradient update over one batch of samples """
        y_pred = self._forward_pass(X)
        loss = np.mean(self.loss_function.loss(y, y_pred))
        acc = self.loss_function.acc(y, y_pred)
        # Calculate the gradient of the loss function wrt y_pred
        loss_grad = self.loss_function.gradient(y, y_pred)
        # Backpropagate. Update weights
        self._backward_pass(loss_grad=loss_grad)

        return loss, acc

    def fit(self, X, y, n_epochs, batch_size):
        """ Trains the model for a fixed number of epochs """
        for _ in self.progressbar(range(n_epochs)):
            
            batch_error = []
            for X_batch, y_batch in batch_iterator(X, y, batch_size=batch_size):
                loss, _ = self.train_on_batch(X_batch, y_batch)
                batch_error.append(loss)

            self.errors["training"].append(np.mean(batch_error))

            if self.val_set is not None:
                val_loss, _ = self.test_on_batch(self.val_set["X"], self.val_set["y"])
                self.errors["validation"].append(val_loss)

        return self.errors["training"], self.errors["validation"]

    def _forward_pass(self, X, training=True):
        """ Calculate the output of the NN """
        layer_output = X
        for layer in self.layers:
            layer_output = layer.forward_pass(layer_output, training)

        return layer_output

    def _backward_pass(self, loss_grad):
        """ Propagate the gradient 'backwards' and update the weights in each layer """
        for layer in reversed(self.layers):
            loss_grad = layer.backward_pass(loss_grad)

    def summary(self, name="Model Summary"):
        # Print model name
        print (AsciiTable([[name]]).table)
        # Network input shape (first layer's input shape)
        print ("Input Shape: %s" % str(self.layers[0].input_shape))
        # Iterate through network and get each layer's configuration
        table_data = [["Layer Type", "Parameters", "Output Shape"]]
        tot_params = 0
        for layer in self.layers:
            layer_name = layer.layer_name()
            params = layer.parameters()
            out_shape = layer.output_shape()
            table_data.append([layer_name, str(params), str(out_shape)])
            tot_params += params
        # Print network configuration table
        print (AsciiTable(table_data).table)
        print ("Total Parameters: %d\n" % tot_params)

    def predict(self, X):
        """ Use the trained model to predict labels of X """
        return self._forward_pass(X, training=False)

In [8]:
class Autoencoder():
    def __init__(self):
        
        self.input_dim = 32
        self.latent_dim = 4 # The dimension of the data embedding

        optimizer = Adam(learning_rate=0.01, b1=0.5)
        loss_function = CrossEntropy

        self.encoder = self.build_encoder(optimizer, loss_function)
        self.decoder = self.build_decoder(optimizer, loss_function)

        self.autoencoder = NeuralNetwork(optimizer=optimizer, loss=loss_function)
        self.autoencoder.layers.extend(self.encoder.layers)
        self.autoencoder.layers.extend(self.decoder.layers)

        print ()
        self.autoencoder.summary(name="RhythNN")

    def build_encoder(self, optimizer, loss_function):

        encoder = NeuralNetwork(optimizer=optimizer, loss=loss_function)
        encoder.add(Dense(16, input_shape=(self.input_dim,)))
        encoder.add(Activation('relu'))
        encoder.add(BatchNormalization(momentum=0.8))
        encoder.add(Dense(8))
        encoder.add(Activation('relu'))
        encoder.add(BatchNormalization(momentum=0.8))
        encoder.add(Dense(self.latent_dim))

        return encoder

    def build_decoder(self, optimizer, loss_function):

        decoder = NeuralNetwork(optimizer=optimizer, loss=loss_function)
        decoder.add(Dense(8, input_shape=(self.latent_dim,)))
        decoder.add(Activation('relu'))
        decoder.add(BatchNormalization(momentum=0.8))
        decoder.add(Dense(16))
        decoder.add(Activation('relu'))
        decoder.add(BatchNormalization(momentum=0.8))
        decoder.add(Dense(self.input_dim))
        decoder.add(Activation('sigmoid'))

        return decoder

In [9]:
dataset = np.loadtxt('data.csv', delimiter=',')

In [10]:
ae = Autoencoder()


+---------+
| RhythNN |
+---------+
Input Shape: (32,)
+----------------------+------------+--------------+
| Layer Type           | Parameters | Output Shape |
+----------------------+------------+--------------+
| Dense                | 528        | (16,)        |
| Activation (ReLU)    | 0          | (16,)        |
| BatchNormalization   | 32         | (16,)        |
| Dense                | 136        | (8,)         |
| Activation (ReLU)    | 0          | (8,)         |
| BatchNormalization   | 16         | (8,)         |
| Dense                | 36         | (4,)         |
| Dense                | 40         | (8,)         |
| Activation (ReLU)    | 0          | (8,)         |
| BatchNormalization   | 16         | (8,)         |
| Dense                | 144        | (16,)        |
| Activation (ReLU)    | 0          | (16,)        |
| BatchNormalization   | 32         | (16,)        |
| Dense                | 544        | (32,)        |
| Activation (Sigmoid) | 0          | (32,)

In [ ]:
print(dataset[1])

In [11]:
ae.autoencoder.fit(dataset,dataset, 30, 16)

Training: 100% [-------------------------------------------------] Time: 0:00:00


([0.6738063440855796,
  0.5909010830439506,
  0.5216445657314778,
  0.4578236664404687,
  0.40595069303146514,
  0.36466669253560247,
  0.3332987622537195,
  0.30990917239423066,
  0.29097820605010244,
  0.27702613508601626,
  0.2649450021534426,
  0.2561516928174888,
  0.2484839673519807,
  0.24248659583113116,
  0.23800250551144783,
  0.23266924112344176,
  0.22752105644161694,
  0.2235107886931343,
  0.219955544052418,
  0.21742440011757846,
  0.21386852456840116,
  0.2113346183930884,
  0.20904211100820974,
  0.20666251451664558,
  0.20412949600954927,
  0.2019157548714623,
  0.19974569291850583,
  0.1976158249267004,
  0.19562125200862646,
  0.19402265980909036],
 [])

In [ ]:
ae2 = Autoencoder()

In [ ]:
ae2.autoencoder.fit(dataset,dataset, 20, 16)

In [ ]:
from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder
from pythonosc import udp_client

# Set up OSC client
ip_address = '127.0.0.1'  # Change this to the IP address of your OSC receiver
port = 9000  # Change this to the port number of your OSC receiver
client = udp_client.SimpleUDPClient(ip_address, port)

In [ ]:

latent_vector = np.array([[1, -4, -2, 5]])

generated_rhythm = ae.decoder.predict(latent_vector)

print(generated_rhythm)

output = []
tolerance=0.5
for op in generated_rhythm[0]:
    if op > tolerance:
        output.append(1)
    else: output.append(0)


print(output)
# client.send_message('/generator', output)

In [ ]:
test_data = dataset[12]
reconstruction = ae.autoencoder.

In [ ]:


from pythonosc.dispatcher import Dispatcher
from pythonosc.osc_server import BlockingOSCUDPServer
import threading
from typing import List, Any

x_value1 = 0
y_value1 = 0
x_value2 = 0
y_value2 = 0
x_value3 = 0
y_value3 = 0
x_value4 = 0
y_value4 = 0
t1 = 0.5
t2 = 0.5
t3 = 0.5
min_output = 0
max_output = 127

# Define the function that will handle the OSC message
def handle_value1(address, *args: List[Any]):
    #MODEL1
    x_value1 = args[1]
    y_value1 = args[2]
    x_value2 = args[4]
    y_value2 = args[5]
    latent_vector = np.array([[x_value1, y_value1,x_value2, y_value2]]) 
    generated_rhythm1 = ae.decoder.predict(latent_vector)
    tolerance1 = args[3]
    output1 = []
    vel1 = []
    for v in generated_rhythm1[0]:
        scaled_value = ((max_output - min_output) * v) + min_output
        scaled_value = round(scaled_value)
        vel1.append(scaled_value)

    for op1 in generated_rhythm1[0]:
        #raw.append(op)
        if op1 > tolerance1:
            output1.append(1)
        else: output1.append(0)

    client.send_message('/generator', output1)
    client.send_message('/vel1', vel1)   
    
def handle_value2(address, *args: List[Any]):
    #MODEL2
    x_value3 = args[1]
    y_value3 = args[2]
    x_value4 = args[4]
    y_value4 = args[5]
    latent_vector2= np.array([[x_value3, y_value3,x_value4, y_value4]])
    generated_rhythm2 = ae2.decoder.predict(latent_vector2)
    tolerance2 = args[3]
    output2 = []
    vel2 = []
    for v2 in generated_rhythm2[0]:
        scaled_value = ((max_output - min_output) * v2) + min_output
        scaled_value = round(scaled_value)
        vel2.append(scaled_value)

    for op2 in generated_rhythm2[0]:
        if op2 > tolerance2:
            output2.append(1)
        else: output2.append(0)
    client.send_message('/generator2', output2)
    client.send_message('/vel2', vel2)
    
    
# def handle_value3(address, *args):
#     #MODEL3
#     x_value3 = args[1]
#     y_value3 = args[2]
#     latent_vector = np.array([[x_value3, y_value3]]) 
#     generated_rhythm3 = generator3.predict(latent_vector)
#     tolerance3 = args[3]
#     output3 = []
#     vel3 = []
#     for v3 in generated_rhythm3[0]:
#         scaled_value = ((max_output - min_output) * v3) + min_output
#         scaled_value = round(scaled_value)
#         vel3.append(scaled_value)

#     for op3 in generated_rhythm3[0]:
#         if op3 > tolerance3:
#             output3.append(1)
#         else: output3.append(0)
#     #print(output)
#     client.send_message('/generator3', output3)
#     client.send_message('/vel3', vel3)
       

    

# Set up the OSC dispatcher and server
dispatcher = Dispatcher()
dispatcher.map("/gen1", handle_value1)
dispatcher.map("/gen2", handle_value2)
# dispatcher.map("/gen3", handle_value3)

server = BlockingOSCUDPServer(("localhost", 9001), dispatcher)

# Start the OSC server in a separate thread
server_thread = threading.Thread(target=server.serve_forever)
server_thread.start()
print('Serving')




In [ ]:
server.server_close()